In [1]:
###test to see if we can implement basic method by Countinglab

import numpy as np
import pandas as pd
import sklearn
from sklearn import linear_model as lm
import os
import csv
from collections import defaultdict
import statsmodels.formula.api as sm
from datetime import datetime

path = os.getcwd()

In [2]:
df = pd.read_csv(path + '/GEFCOM/mod_data.csv')

def season(row):
    if row['month'] < 3:
        val = 'winter'
    elif row['month'] < 6:
        val = 'spring'
    elif row['month'] < 9:
        val = 'summer'
    elif row['month'] < 12:
        val = 'fall'
    else:
        val = 'winter'
        
    return val

df['season'] = df.apply(season, axis=1)df

#create interaction terms
for i in range(1, 12):
    t2 = 'w{}2'.format(i)
    td = 'w{}d'.format(i)
    t2d = 'w{}2d'.format(i)
    df[t2] = df['w{}'.format(i)]**2
    df[td] = df['w{}'.format(i)] * df['totDay']
    df[t2d] = df['w{}'.format(i)]**2 * df['totDay']

In [3]:
dataframes = defaultdict(dict)
###split up into 3840 dataframes
tot = 0
for seas, df_season in df.groupby('season'):
    for weekday, df_weekday in  df_season.groupby('weekday'):
        dataframes[seas][weekday] = defaultdict(dict)
        for zone, df_zone in df_weekday.groupby('zone'):
            for hour, df_hour in df_zone.groupby('hour'):
                dataframes[seas][weekday][zone][hour] = df_hour

In [4]:
bestWeather = defaultdict(dict)
bestModel = defaultdict(dict)
counter = 0
for seas, valSeas in dataframes.items():
    for wd, valWd in valSeas.items():
        bestWeather[seas][wd] = defaultdict(dict)
        bestModel[seas][wd] = defaultdict(dict)
        for zone, valZone in valWd.items():
            for hr, dataToFit in valZone.items():
                counter += 1
                #print (zone)
                #if counter % 50 == 0:
                #    print (counter)
                for i in range(1, 12):
                    t2 = 'w{}2'.format(i)
                    td = 'w{}d'.format(i)
                    t2d = 'w{}2d'.format(i)
                    formula = 'load ~ w{} + w{}2 + w{}d + w{}2d + totDay'.format(i,i,i,i)
                    mod = sm.ols(formula, data = dataToFit)
                    res = mod.fit()
                    if i == 1:
                        bestModel[seas][wd][zone][hr] = res
                        bestWeather[seas][wd][zone][hr] = i
                        mse = res.mse_resid
                    else:
                        #print (mse, res.mse_resid)
                        if res.mse_resid < mse:
                            bestModel[seas][wd][zone][hr] = res
                            bestWeather[seas][wd][zone][hr] = i
                            mse = res.mse_resid
        

In [5]:
#for seas, valSeas in dataframes.items():
#    for wd, valWd in valSeas.items():
#        for zone, valZone in valWd.items():
#            for hr, dataToFit in valZone.items():
#                if bestWeather[seas][wd][zone][hr] != 2:
#                    print (bestWeather[seas][wd][zone][hr])

In [8]:
#read in solutions
solutions = pd.read_csv(path + '/GEFCOM/Load_solution.csv')
solutions = solutions[:][0:1175] #remove forecast values
solutions['season'] = solutions.apply(season, axis=1)


In [136]:
#create a dict that stores the times and loads of each zone
temps  = {}
for i in range(1, 12):
    #keys for the temperature
    temps['temp_{}'.format(i)] = [] 
    #keys for the time (datetime objects)
    temps['time_{}'.format(i)] = []


with open(path + '/GEFCOM/temperature_history.csv', 'rt') as tcsv:
    tempreader = csv.reader(tcsv)
    for index, row in list(enumerate(tempreader)):
        #skip first row
        if index == 0:
            continue
        for i, col in enumerate(row):
           #ignore empty entries
            if col == '':
                continue
           #we look at the loads and temperatures which 
           #start in the 5th column
            if i > 3:
                hour = i - 4
                year = int(row[1])
                month = int(row[2])
                day = int(row[3])
                timeVal = datetime(year, month, day, hour)
                temps['time_{}'.format(row[0])].append(timeVal)
                temps['temp_{}'.format(row[0])].append(int(col))

In [138]:
dates = set()
seen = {}

for i in range(len(solutions['id'])):
    year = int(solutions['year'][i])
    month = int(solutions['month'][i])
    day = int(solutions['day'][i])
    if (year, month, day) in seen:
        continue
    seen[(year, month, day)] = True
    for j in range(24):
        dates.add((year, month, day, j))

In [139]:
tempvals = {}
for i in range(len(temps['time_1'])):
    if len(dates) == 0:
        break
    year = temps['time_1'][i].year
    if year == 2004 or year == 2007 or year == 2008:
        continue
    month = temps['time_1'][i].month
    day = temps['time_1'][i].day
    hour = temps['time_1'][i].hour
    val = (year, month, day, hour)
    
    if val in dates:
        for j in range(1, 12):
            header = 'temp_{}'.format(j)
            tempvals[(year, month, day, hour, j)] = temps[header][i]
        dates.remove(val)
    
    

In [165]:
#evaluate our models
sampleSize = int(1175* 0.25)
sample = np.random.randint(0, 1175, sampleSize)
minDate = datetime(2004, 1, 1)
num = 0
denom = 0
for i in sample:
    #pull out relevant indicators
    s = solutions['season'][i] #get the season
    z = solutions['zone_id'][i] #get the zone number
    if z == 21:
        continue
    date = datetime(solutions['year'][i], solutions['month'][i], solutions['day'][i])
    d = (date - minDate).days #total days
    wd = (1 if date.weekday() < 5 else 0)
    for j in range(24):
        hr = 'h{}'.format(j + 1)
        val = solutions[hr][i]
        model = bestModel[s][wd][z][j]
        wIndex = bestWeather[s][wd][z][j]
        T = tempvals[(date.year, date.month, date.day, j, wIndex)]
        inpt = [1, T, T**2, T*d, T**2*d, d]
        pred = sum(inpt[i]*model.params[i] for i in range(len(inpt)))
        num += (pred - val)**2 * 1 #solutions['weight'][i]
        denom += 1 #solutions['weight'][i]
        
print('WMSE is: ', (num/denom)**0.5)

WMSE is:  10264.7652241


In [167]:
list(df)

['id',
 'zone',
 'year',
 'month',
 'day',
 'totDay',
 'hour',
 'weekday',
 'w1',
 'w2',
 'w3',
 'w4',
 'w5',
 'w6',
 'w7',
 'w8',
 'w9',
 'w10',
 'w11',
 'load',
 'season',
 'w12',
 'w1d',
 'w12d',
 'w22',
 'w2d',
 'w22d',
 'w32',
 'w3d',
 'w32d',
 'w42',
 'w4d',
 'w42d',
 'w52',
 'w5d',
 'w52d',
 'w62',
 'w6d',
 'w62d',
 'w72',
 'w7d',
 'w72d',
 'w82',
 'w8d',
 'w82d',
 'w92',
 'w9d',
 'w92d',
 'w102',
 'w10d',
 'w102d',
 'w112',
 'w11d',
 'w112d']